### Setting up the Data

In [6]:
# import numpy and pandas libraries
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier 

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import matplotlib.pyplot as plt

from sklearn import datasets
# set random seed to ensure that results are repeatable
np.random.seed(1)

In [7]:
X_test = pd.read_csv("C:/Users/18137/rfm_test_X.csv")
y_test= pd.read_csv("C:/Users/18137/rfm_test_y.csv")
X_train= pd.read_csv("C:/Users/18137/rfm_train_X.csv")
y_train= pd.read_csv("C:/Users/18137/rfm_train_y.csv")

In [8]:
X_train

,recency,monetary,frequency,recency_score,frequency_score,monetary_score
0,9,142,1419.73,5,4,5
1,5,152,1026.42,5,4,5
2,7,16,266.39,5,2,2
3,306,210,3724.77,1,5,5
4,35,34,1524.46,3,4,3
...,...,...,...,...,...,...
3155,40,255,2087.88,3,5,5
3156,16,19,388.52,4,2,2
3157,227,24,413.46,1,2,2
3158,218,1,179.00,1,1,1



### Modeling the Data

In [9]:
performance = pd.DataFrame({"model": [], "Accuracy": [], "Precision": [], "Recall": [], "F1": []})

### Logistic Regression Model

In [10]:
log_reg_model = LogisticRegression()
_ = log_reg_model.fit(X_train, np.ravel(y_train))

C:\Users\18137\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [11]:
model_preds = log_reg_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"default logistic", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.703125,0.697479,0.976471,0.813725


### Logistic Regression Model(RandomizedSearchCV )

In [12]:
score_measure = "Precision"
LR=LogisticRegression()
kfolds = 4
param_grid = {'C': [0.1, 0.5, 10,0.001], 
              "solver" : [ 'lbfgs', 'liblinear'],
              "penalty" : ['l1','l2','elastic'],
             "warm_start":[True, False]} 
  
grid = RandomizedSearchCV(LR, param_grid, verbose = 1, n_jobs = -1)
  
# fitting the model for grid search
grid.fit(X_train, np.ravy_train)
print(f"The best {score_measure} score is {grid.best_score_}")
print(f"... with parameters: {grid.best_params_}")

bestRecallTree = grid.best_estimator_

AttributeError: module 'numpy' has no attribute 'ravy_train'

In [13]:
model_preds = grid.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Logistic Regression Randomised", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

NotFittedError: This RandomizedSearchCV instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

### Logistic Regression Model(GridSearchCV)

In [18]:
score_measure = "precision"
kfolds = 3
param_grid = {'C': [1,5,0.5], 
              'solver' : [ 'liblinear'],
              'penalty' : ['l1']} 
  
grid = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3)
  
# fitting the model for grid search
grid.fit(X_train, y_train)
print(f"The best {score_measure} score is {grid.best_score_}")
print(f"... with parameters: {grid.best_params_}")

bestRecallTree = grid.best_estimator_

Fitting 5 folds for each of 3 candidates, totalling 15 fits


C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 1/5] END .C=1, penalty=l1, solver=liblinear;, score=0.946 total time=   0.6s


C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 2/5] END .C=1, penalty=l1, solver=liblinear;, score=0.937 total time=   0.6s


...

[CV 4/5] END C=0.5, penalty=l1, solver=liblinear;, score=0.927 total time=   0.3s


C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[CV 5/5] END C=0.5, penalty=l1, solver=liblinear;, score=0.926 total time=   0.3s


C:\Users\18137\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


The best precision score is 0.9528481012658225
... with parameters: {'C': 5, 'penalty': 'l1', 'solver': 'liblinear'}


In [14]:
model_preds = grid.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Logistic Regression Grid", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])

NotFittedError: This RandomizedSearchCV instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [15]:
performance.sort_values(by =['Recall'])

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.703125,0.697479,0.976471,0.813725


### Decision Tree Model

In [21]:
from sklearn.tree import DecisionTreeClassifier

# instantiate the DecisionTreeClassifier model 
clf = DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=0)
# fit the model
clf.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=0)

### Decision tree model (Randomsearch)

In [22]:
score_measure = "recall"
kfolds = 4

param_grid = {
    'min_samples_split': np.arange(1,80),  
    'min_samples_leaf': np.arange(1,40),
    'min_impurity_decrease': np.arange(0.0001, 0.01, 0.0005),
    'max_leaf_nodes': np.arange(5, 200), 
    'max_depth': np.arange(1,50), 
    'criterion': ['entropy', 'gini'],
}

dtree = DecisionTreeClassifier()
rand_search = RandomizedSearchCV(estimator = dtree, param_distributions=param_grid, cv=kfolds, n_iter=500,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = rand_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestRecallTree = rand_search.best_estimator_

Fitting 4 folds for each of 500 candidates, totalling 2000 fits
The best recall score is nan
... with parameters: {'min_samples_split': 3, 'min_samples_leaf': 27, 'min_impurity_decrease': 0.0041, 'max_leaf_nodes': 87, 'max_depth': 39, 'criterion': 'entropy'}


C:\Users\18137\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
16 fits failed out of a total of 2000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
16 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\18137\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\18137\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 937, in fit
    super().fit(
  File "C:\Users\18137\anaconda3\lib\site-packages\sklearn\tree\_classes.py", line 250, in fit
    raise ValueError(
ValueError: min_samples_split must be an integer greater than 1 or a float in (

In [23]:
c_matrix = confusion_matrix(y_test, rand_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Random search", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.703125,0.697479,0.976471,0.813725
0,Logistic Regression Randomised,1.000000,1.000000,1.000000,1.000000
0,Logistic Regression Randomised,1.000000,1.000000,1.000000,1.000000
0,Logistic Regression Grid,1.000000,1.000000,1.000000,1.000000
0,Random search,1.000000,1.000000,1.000000,1.000000


### Decision tree model (Gridsearch)

In [24]:
score_measure = "recall"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(45,55),  
    'min_samples_leaf': np.arange(27,32),
    'min_impurity_decrease': np.arange(0.004, 0.005, 0.0001),
    'max_leaf_nodes': np.arange(70,75), 
    'max_depth': np.arange(32,36), 
    'criterion': ['gini'],
}

dtree = DecisionTreeClassifier()
grid_search = GridSearchCV(estimator = dtree, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestRecallTree = grid_search.best_estimator_

Fitting 5 folds for each of 10000 candidates, totalling 50000 fits
The best recall score is nan
... with parameters: {'criterion': 'gini', 'max_depth': 32, 'max_leaf_nodes': 70, 'min_impurity_decrease': 0.004, 'min_samples_leaf': 27, 'min_samples_split': 45}


C:\Users\18137\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the test scores are non-finite: [nan nan nan ... nan nan nan]
  warnings.warn(
C:\Users\18137\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:969: UserWarning: One or more of the train scores are non-finite: [nan nan nan ... nan nan nan]
  warnings.warn(


In [25]:
c_matrix = confusion_matrix(y_test, grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Grid search", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.703125,0.697479,0.976471,0.813725
0,Logistic Regression Randomised,1.000000,1.000000,1.000000,1.000000
0,Logistic Regression Randomised,1.000000,1.000000,1.000000,1.000000
0,Logistic Regression Grid,1.000000,1.000000,1.000000,1.000000
0,Random search,1.000000,1.000000,1.000000,1.000000
0,Grid search,1.000000,1.000000,1.000000,1.000000


# SVM Classification model(Linear Kernel)

In [26]:
svm_lin_model = SVC(kernel="linear", probability=True)
_ = svm_lin_model.fit(X_train, np.ravel(y_train))

In [27]:
model_preds = svm_lin_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"svm with linear kernel", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.703125,0.697479,0.976471,0.813725
0,Logistic Regression Randomised,1.000000,1.000000,1.000000,1.000000
0,Logistic Regression Randomised,1.000000,1.000000,1.000000,1.000000
0,Logistic Regression Grid,1.000000,1.000000,1.000000,1.000000
0,Random search,1.000000,1.000000,1.000000,1.000000
0,Grid search,1.000000,1.000000,1.000000,1.000000
0,svm with linear kernel,1.000000,1.000000,1.000000,1.000000


# SVM Classification model(RBF Kernel)

In [28]:
svm_rbf_model = SVC(kernel="rbf", C=10, gamma='scale', probability=True)
_ = svm_rbf_model.fit(X_train, np.ravel(y_train))

In [29]:
model_preds = svm_rbf_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"svm with rbf kernel", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.703125,0.697479,0.976471,0.813725
0,Logistic Regression Randomised,1.000000,1.000000,1.000000,1.000000
0,Logistic Regression Randomised,1.000000,1.000000,1.000000,1.000000
0,Logistic Regression Grid,1.000000,1.000000,1.000000,1.000000
0,Random search,1.000000,1.000000,1.000000,1.000000
0,Grid search,1.000000,1.000000,1.000000,1.000000
0,svm with linear kernel,1.000000,1.000000,1.000000,1.000000
0,svm with rbf kernel,1.000000,1.000000,1.000000,1.000000


# SVM Classification model(Polynomial Kernel)

In [32]:
svm_poly_model = SVC(kernel="poly", degree=3, coef0=1, C=10, probability=True)
_ = svm_poly_model.fit(X_train, np.ravel(y_train))

In [33]:
model_preds = svm_poly_model.predict(X_test)
c_matrix = confusion_matrix(y_test, model_preds)
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"svm with polynomial kernel", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,default logistic,0.703125,0.697479,0.976471,0.813725
0,Logistic Regression Randomised,1.000000,1.000000,1.000000,1.000000
0,Logistic Regression Randomised,1.000000,1.000000,1.000000,1.000000
0,Logistic Regression Grid,1.000000,1.000000,1.000000,1.000000
0,Random search,1.000000,1.000000,1.000000,1.000000
0,Grid search,1.000000,1.000000,1.000000,1.000000
0,svm with linear kernel,1.000000,1.000000,1.000000,1.000000
0,svm with rbf kernel,1.000000,1.000000,1.000000,1.000000
0,svm with polynomial kernel,1.000000,1.000000,1.000000,1.000000
0,svm with polynomial kernel,1.000000,1.000000,1.000000,1.000000


#### As we can see from the results, all the models have been overfitted a part from the default logistic model. I have considered the rfm scores for segmentation of the customers. Here I have considered the segment to be the target variable, I have measured the segment using rfm scores which are calculated separately here. 

# Neural Networks

In [16]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score

In [17]:
%%time

ann = MLPClassifier(hidden_layer_sizes=(60,50,40), solver='adam', max_iter=200)
_ = ann.fit(X_train, y_train)

C:\Users\18137\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Wall time: 2.96 s


In [18]:
%%time
y_pred = ann.predict(X_test)

Wall time: 6.6 ms


In [19]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.80      0.65      0.72        57
           1       0.61      0.91      0.73       108
           2       0.64      0.62      0.63        26
           3       0.78      0.92      0.85       106
           4       0.92      0.80      0.85       190
           5       0.80      0.79      0.80       139
           6       0.74      0.30      0.43        46
           7       1.00      0.33      0.50         9
           8       0.84      0.87      0.85        94
           9       0.70      0.47      0.56        15

    accuracy                           0.78       790
   macro avg       0.78      0.67      0.69       790
weighted avg       0.80      0.78      0.77       790



## Neural Network with RandomizedSearchCV

In [27]:
%%time

score_measure = "accuracy"
kfolds = 5

param_grid = {
    'hidden_layer_sizes': [ (50,), (70,),(50,30), (40,20), (60,40, 20), (70,50,40)],
    'activation': ['tanh', 'relu'],
    'solver': ['adam', 'sgd'],
    'alpha': [0, .2, .5, .7, 1],
    'learning_rate': ['constant', 'invscaling', 'adaptive'],
    'learning_rate_init': [0.001, 0.01, 0.1, 0.2, 0.5],
    'max_iter': [5000]
}

ann = MLPClassifier()
grid_search = RandomizedSearchCV(estimator = ann, param_distributions=param_grid, cv=kfolds, n_iter=100,
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

bestRecallTree = grid_search.best_estimator_

print(grid_search.best_params_)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


C:\Users\18137\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'solver': 'adam', 'max_iter': 5000, 'learning_rate_init': 0.001, 'learning_rate': 'invscaling', 'hidden_layer_sizes': (50, 30), 'alpha': 0.2, 'activation': 'tanh'}
Wall time: 5min 54s


In [28]:
%%time
y_pred = bestRecallTree.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.96      0.96        57
           1       0.98      0.83      0.90       108
           2       0.68      0.73      0.70        26
           3       0.79      0.98      0.88       106
           4       0.95      1.00      0.98       190
           5       0.94      0.79      0.86       139
           6       0.93      0.87      0.90        46
           7       1.00      1.00      1.00         9
           8       0.94      0.98      0.96        94
           9       0.87      0.87      0.87        15

    accuracy                           0.91       790
   macro avg       0.90      0.90      0.90       790
weighted avg       0.92      0.91      0.91       790

Wall time: 8.78 ms


SGD didn't reach the global minima. More hyperparametertuning can be done in the future. The accuracy for Neural network is 78%.